<a href="https://colab.research.google.com/github/Tiru28/ModernAIPro/blob/main/Moving_to_similarity_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q langchain-community chromadb sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 75.1 MB/s eta 0:

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
documents = [
        "Biryani is a fragrant rice dish cooked with marinated meat, typically chicken or mutton, and seasoned with a variety of spices such as saffron and turmeric. It originated among the Muslim communities of the Indian subcontinent.",
        "Masala Dosa is a South Indian specialty consisting of a crispy, rice batter crepe filled with a spicy mix of mashed potatoes. It's served with a trio of chutneys and sambar, a lentil-based stew.",
        "Rogan Josh is a type of curry hailing from Kashmir. It features braised lamb chunks cooked with a gravy based on browned onions, yogurt, garlic, ginger, and aromatic spices like cloves, bay leaves, cardamom, and cinnamon.",
        "Pani Puri, also known as Golgappa, is a popular street snack that consists of a round, hollow puri filled with a mixture of flavored water (pani), tamarind chutney, chili, chaat masala, potato, onion, and chickpeas.",
        "Palak Paneer is a North Indian curry made from spinach and paneer (Indian cottage cheese) in a thick curry sauce based on pureed spinach, seasoned with garlic, garam masala, and other spices.",
        "Aloo Gobi is a vegetarian dish from the Indian subcontinent made with potatoes (aloo), cauliflower (gobi), and Indian spices. It is yellowish in color due to the use of turmeric, and sometimes contains kalonji and curry leaves.",
        "Samosas are a popular teatime snack and are often served as an appetizer. They are deep-fried pastries with a savory filling, such as spiced potatoes, onions, peas, and lentils.",
        "Butter Chicken is a rich and creamy North Indian curry made with marinated chicken cooked in a tomato-based sauce with butter, cream, and a blend of spices. It is often served with naan or rice.",
        "Chole Bhature is a combination of spicy chickpea curry and deep-fried bread called bhature. This North Indian dish is often enjoyed for breakfast or lunch and is served with onions, pickles, and yogurt.",
        "Rasam is a South Indian soup made from tamarind juice, tomatoes, and a blend of spices including mustard seeds, curry leaves, and black pepper. It is typically served with rice.",
        "Dhokla is a steamed savory cake made from fermented rice and chickpea batter. It is a popular snack from Gujarat and is often garnished with mustard seeds, green chilies, and coriander.",
        "Pav Bhaji is a Mumbai street food consisting of a spicy vegetable mash (bhaji) served with buttered bread rolls (pav). It is garnished with chopped onions, cilantro, and a squeeze of lemon.",
        "Vindaloo is a Goan curry known for its fiery heat and tangy flavor. It is made with marinated pork cooked in a sauce of vinegar, garlic, and a variety of spices including red chilies and cinnamon.",
        "Dum Aloo is a popular North Indian dish where potatoes are cooked slowly in a rich, spiced yogurt gravy. It is often garnished with coriander and served with rice or bread.",
        "Idli is a South Indian breakfast dish made from steamed fermented rice and lentil batter. It is typically served with coconut chutney and sambar."
]

In [ ]:
from langchain.docstore.document import Document
doc = []

for row in documents:
    page = Document(page_content=row)
    doc.append(page)

doc

[Document(metadata={}, page_content='Biryani is a fragrant rice dish cooked with marinated meat, typically chicken or mutton, and seasoned with a variety of spices such as saffron and turmeric. It originated among the Muslim communities of the Indian subcontinent.'),
 Document(metadata={}, page_content="Masala Dosa is a South Indian specialty consisting of a crispy, rice batter crepe filled with a spicy mix of mashed potatoes. It's served with a trio of chutneys and sambar, a lentil-based stew."),
 Document(metadata={}, page_content='Rogan Josh is a type of curry hailing from Kashmir. It features braised lamb chunks cooked with a gravy based on browned onions, yogurt, garlic, ginger, and aromatic spices like cloves, bay leaves, cardamom, and cinnamon.'),
 Document(metadata={}, page_content='Pani Puri, also known as Golgappa, is a popular street snack that consists of a round, hollow puri filled with a mixture of flavored water (pani), tamarind chutney, chili, chaat masala, potato, onio

In [ ]:
from langchain_community.vectorstores import Chroma
vector_db3 = Chroma.from_documents(doc, embeddings)

In [ ]:
def language_chat(message, history):
  results = vector_db3.similarity_search(message, k=1)
  shortened_results = [doc.page_content.split(".")[0] for doc in results]
  return "\n".join(shortened_results)

In [ ]:
import gradio as gr
demo = gr.ChatInterface(
    language_chat, title="Vector DB Search", theme='taithrah/minimal')
demo.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ea5fed09a18bd9a1ad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
